In [154]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import datetime
from datetime import date, datetime
import numpy as np
import random

In [155]:
current_season = int(input('What is the current season?'))

In [156]:
if not os.getcwd().endswith('Football Forecasting Version 2'):
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

dir_path = os.getcwd()

In [157]:
# CREATES DICTIONARY OF THREE LETTER ON CLUBS

res = []
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a folder
    if os.path.isdir(os.path.join(dir_path, path)) and 'Matchweek' in path:
        res.append(path)

three_let_dict = {}
for matchweek_folder in res:
    os.chdir(fr'{dir_path}\{matchweek_folder}')
    a = pd.read_csv(random.choice(os.listdir(fr'{dir_path}\{matchweek_folder}')))
    for index, row in a.iterrows():
        three_let_dict[row['Club'].strip()] =  row['Code']
    os.chdir(dir_path)


three_let_dict['Tottenham'] = three_let_dict.pop('Tottenham Hotspur')
three_let_dict['Brighton'] = three_let_dict.pop('Brighton and Hove Albion')
three_let_dict['Manchester Utd'] = three_let_dict.pop('Manchester United')
three_let_dict['Newcastle Utd'] = three_let_dict.pop('Newcastle United')
three_let_dict['West Ham'] = three_let_dict.pop('West Ham United')
three_let_dict['Wolves'] = three_let_dict.pop('Wolverhampton Wanderers')
three_let_dict["Nott'ham Forest"] = three_let_dict.pop('Nottingham Forest')
three_let_dict["Sheffield Utd"] = three_let_dict.pop('Sheffield United')
three_let_dict["Huddersfield"] = three_let_dict.pop('Huddersfield Town')
three_let_dict["West Brom"] = three_let_dict.pop('West Bromwich Albion')

In [158]:
### FIXTURE LIST

res = []
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a folder
    if os.path.isdir(os.path.join(dir_path, path)) and 'Fixture' in path:
        res.append(path)
print (res)
df = pd.DataFrame()
for fixtures in res:
    os.chdir(fr'{dir_path}\{fixtures}')
    folder = os.getcwd()
    list_of_files = os.listdir()
    for i in list_of_files:
        b = pd.read_csv(fr'{folder}\{i}', index_col = 0)
        b['Date'] = pd.to_datetime(b['Date'])
        b['Club'] = b['Club'].map(three_let_dict)
        b['Opponent'] = b['Opponent'].map(three_let_dict)
        df = pd.concat([df,b])
    os.chdir(dir_path)
df = df.reset_index(drop = True)
df = df.sort_values(by = ['Date'] )
df = df.dropna()

['2017_2018_Fixture', '2018_2019_Fixture', '2019_2020_Fixture', '2020_2021_Fixture', '2021_2022_Fixture', '2022_2023_Fixture']


In [159]:
### SPI DATA

res = []
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a folder
    if os.path.isdir(os.path.join(dir_path, path)) and 'SPI' in path:
        res.append(path)
print (res)
df1 = pd.DataFrame()
for SPIdata in res:
    os.chdir(fr'{dir_path}\{SPIdata}')
    folder = os.getcwd()
    list_of_files = os.listdir()
    for i in list_of_files:
        b = pd.read_csv(fr'{folder}\{i}', index_col=0)
        b['date'] = pd.to_datetime(b['date'])
        b['Season'] = int(i[:4])
        b['team'] = b['team'].str.replace('Leicester', 'Leicester City')
        b['team'] = b['team'].str.replace('Newcastle', 'Newcastle Utd')
        b['team'] = b['team'].str.replace('Southamon', 'Southampton')
        b['team'] = b['team'].str.replace('Norwich', 'Norwich City')
        b['team'] = b['team'].str.replace('Man City', 'Manchester City')
        b['team'] = b['team'].str.replace('Man United', 'Manchester Utd')
        b['team'] = b['team'].str.replace('Nottm Forest', "Nott'ham Forest")
        b['team'] = b['team'].map(three_let_dict)
        df1 = pd.concat([df1,b])
    os.chdir(dir_path)
df1 = df1.reset_index(drop = True)
df1 = df1.sort_values(by = ['date'] )


['2017_2018_SPIdata', '2018_2019_SPIdata', '2019_2020_SPIdata', '2020_2021_SPIdata', '2021_2022_SPIdata', '2022_2023_SPIdata']


In [160]:
res = []
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a folder
    if os.path.isdir(os.path.join(dir_path, path)) and 'Matchweek' in path:
        res.append(path)

df2 = pd.DataFrame()
for matchweek in res:
    os.chdir(fr'{dir_path}\{matchweek}')
    folder = os.getcwd()
    list_of_files = os.listdir()
    for i in list_of_files:
            a = pd.read_csv(fr'{folder}\{i}', index_col = 0)
            a['Season'] = int(matchweek[:4])
            a['Last Match Date'] = pd.to_datetime(a['Last Match Date'])
            a = a.drop (columns = ['Club','W','D','L','GF', 'GA', 'GD'])
            df2 = pd.concat([df2,a])
    os.chdir(dir_path)

df2 = df2.sort_values(by = ['Last Match Date'])

In [161]:
fixture_fixture_merge  = pd.merge_asof(left=df,right=df,left_on = ['Date'], right_on = ['Date'], left_by = ['Club'], right_by = ['Opponent'], direction='nearest')
fixture_fixture_merge = fixture_fixture_merge.drop(columns = ['Venue_y', 'Result_y','Opponent_y','Club_y'])

In [162]:
matchweek_fixture_merge_1 = pd.merge_asof(left=fixture_fixture_merge ,right=df2,left_on = ['Date'], right_on = ['Last Match Date'], left_by = ['Club_x'], right_by = ['Code'], direction='nearest')
matchweek_fixture_merge_1 = matchweek_fixture_merge_1.drop (columns = ['Last Match Date', 'Code']) 
matchweek_fixture_merge_2 = pd.merge_asof(left=matchweek_fixture_merge_1,right=df2,left_on = ['Date'], right_on = ['Last Match Date'], left_by = ['Opponent_x'], right_by = ['Code'], direction='nearest')
matchweek_fixture_merge_2 = matchweek_fixture_merge_2.drop (columns = ['Season_x','Last Match Date', 'Code']) 


In [163]:
spi_everything_merge_1 = pd.merge_asof(left=matchweek_fixture_merge_2, right=df1, left_on = ['Date'], right_on = ['date'], left_by = ['Club_x'], right_by = ['team'], direction='nearest')
spi_everything_merge_1 = spi_everything_merge_1.drop (columns = ['team','date', 'Season_y']) 
spi_everything_merge_2 = pd.merge_asof(left=spi_everything_merge_1 , right=df1, left_on = ['Date'], right_on = ['date'], left_by = ['Opponent_x'], right_by = ['team'], direction='nearest')
spi_everything_merge_2 = spi_everything_merge_2.drop (columns = ['team','date', 'Season_y', 'GF_y', 'GA_y', 'xG_y', 'xGA_y']) 

In [164]:
rename_columns = {'Venue_x' : 'Venue', 'Result_x' : 'Result', 'Club_x':'Club', 'Season_x' : 'Season', 'Opponent_x': 'Opp'}

df_final = spi_everything_merge_2.rename(rename_columns, axis = 'columns')
df_final = df_final.sort_values(by = ['Club','Season','Date'])
display(df_final)


,Date,Venue,Result,GF_x,GA_x,Opp,xG_x,xGA_x,Poss_x,Club,...,Pl_y,Pts_y,Form_y,spi_x,off_x,def_x,Season,spi_y,off_y,def_y
0,2017-08-11,Home,W,4.0,3.0,LEI,2.5,1.5,68.0,ARS,...,1,0,0.000000,78.1,2.6,0.9,2017,62.1,1.8,1.1
27,2017-08-19,Away,L,0.0,1.0,STK,1.5,0.7,76.0,ARS,...,2,3,1.500000,78.0,2.5,0.9,2017,60.7,1.6,0.9
58,2017-08-27,Away,L,0.0,4.0,LIV,0.6,3.1,52.0,ARS,...,3,7,2.333333,76.0,2.5,0.9,2017,78.5,2.4,0.8
73,2017-09-09,Home,W,3.0,0.0,BOU,2.2,0.6,58.0,ARS,...,4,0,0.000000,76.7,2.5,0.9,2017,54.3,1.6,1.2
98,2017-09-17,Away,D,0.0,0.0,CHE,1.4,0.8,49.0,ARS,...,5,10,2.000000,77.6,2.4,0.8,2017,82.6,2.5,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,2023-02-11,Away,W,2.0,1.0,SOU,0.8,1.1,59.0,WOL,...,22,15,0.600000,65.1,1.6,0.8,2022,60.2,1.7,1.0
4249,2023-02-18,Home,L,0.0,1.0,BOU,1.2,1.0,67.0,WOL,...,23,21,1.000000,64.9,1.6,0.8,2022,56.1,1.6,1.1
4263,2023-02-24,Away,D,1.0,1.0,FUL,0.8,0.5,49.0,WOL,...,25,39,1.600000,64.6,1.6,0.8,2022,68.9,1.8,0.8
4280,2023-03-01,Away,L,0.0,2.0,LIV,0.8,2.5,43.0,WOL,...,24,39,2.000000,64.8,1.6,0.8,2022,82.9,2.5,0.7


In [165]:
df_final = df_final.sort_values(by = ['Club','Season','Date'])
df_final['Pl_x'] = df_final.groupby(['Club','Season'])['Pl_x'].shift(1)
df_final['Position_x'] = df_final.groupby(['Club','Season'])['Position_x'].shift(1)
df_final['Pts_x'] = df_final.groupby(['Club','Season'])['Pts_x'].shift(1)
df_final['Form_x'] = df_final.groupby(['Club','Season'])['Pts_x'].shift(1)



df_final = df_final.sort_values(by = ['Opp','Season','Date'])
df_final['Pl_y'] = df_final.groupby(['Opp','Season'])['Pl_y'].shift(1)
df_final['Position_y'] = df_final.groupby(['Opp','Season'])['Position_y'].shift(1)
df_final['Pts_y'] = df_final.groupby(['Opp','Season'])['Pts_y'].shift(1)
df_final['Form_y'] = df_final.groupby(['Opp','Season'])['Pts_y'].shift(1)

df_final = df_final.sort_values(by = ['Club','Season','Date'])

df_final = df_final.dropna()


,Date,Venue,Result,GF_x,GA_x,Opp,xG_x,xGA_x,Poss_x,Club,...,Pl_y,Pts_y,Form_y,spi_x,off_x,def_x,Season,spi_y,off_y,def_y
0,2017-08-11,Home,W,4.0,3.0,LEI,2.5,1.5,68.0,ARS,...,1,0,0.000000,78.1,2.6,0.9,2017,62.1,1.8,1.1
27,2017-08-19,Away,L,0.0,1.0,STK,1.5,0.7,76.0,ARS,...,2,3,1.500000,78.0,2.5,0.9,2017,60.7,1.6,0.9
58,2017-08-27,Away,L,0.0,4.0,LIV,0.6,3.1,52.0,ARS,...,3,7,2.333333,76.0,2.5,0.9,2017,78.5,2.4,0.8
73,2017-09-09,Home,W,3.0,0.0,BOU,2.2,0.6,58.0,ARS,...,4,0,0.000000,76.7,2.5,0.9,2017,54.3,1.6,1.2
98,2017-09-17,Away,D,0.0,0.0,CHE,1.4,0.8,49.0,ARS,...,5,10,2.000000,77.6,2.4,0.8,2017,82.6,2.5,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,2023-02-11,Away,W,2.0,1.0,SOU,0.8,1.1,59.0,WOL,...,22,15,0.600000,65.1,1.6,0.8,2022,60.2,1.7,1.0
4249,2023-02-18,Home,L,0.0,1.0,BOU,1.2,1.0,67.0,WOL,...,23,21,1.000000,64.9,1.6,0.8,2022,56.1,1.6,1.1
4263,2023-02-24,Away,D,1.0,1.0,FUL,0.8,0.5,49.0,WOL,...,25,39,1.600000,64.6,1.6,0.8,2022,68.9,1.8,0.8
4280,2023-03-01,Away,L,0.0,2.0,LIV,0.8,2.5,43.0,WOL,...,24,39,2.000000,64.8,1.6,0.8,2022,82.9,2.5,0.7


,Date,Venue,Result,GF_x,GA_x,Opp,xG_x,xGA_x,Poss_x,Club,...,Pl_y,Pts_y,Form_y,spi_x,off_x,def_x,Season,spi_y,off_y,def_y
58,2017-08-27,Away,L,0.0,4.0,LIV,0.6,3.1,52.0,ARS,...,2.0,4.0,1.0,76.0,2.5,0.9,2017,78.5,2.4,0.8
73,2017-09-09,Home,W,3.0,0.0,BOU,2.2,0.6,58.0,ARS,...,3.0,0.0,0.0,76.7,2.5,0.9,2017,54.3,1.6,1.2
98,2017-09-17,Away,D,0.0,0.0,CHE,1.4,0.8,49.0,ARS,...,4.0,9.0,6.0,77.6,2.4,0.8,2017,82.6,2.5,0.6
119,2017-09-25,Home,W,2.0,0.0,WBA,2.2,0.9,69.0,ARS,...,5.0,8.0,7.0,78.2,2.4,0.8,2017,54.9,1.4,1.0
137,2017-10-01,Home,W,2.0,0.0,BHA,2.4,0.4,64.0,ARS,...,6.0,7.0,4.0,79.1,2.5,0.8,2017,47.8,1.2,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,2023-02-11,Away,W,2.0,1.0,SOU,0.8,1.1,59.0,WOL,...,21.0,15.0,15.0,65.1,1.6,0.8,2022,60.2,1.7,1.0
4249,2023-02-18,Home,L,0.0,1.0,BOU,1.2,1.0,67.0,WOL,...,22.0,18.0,17.0,64.9,1.6,0.8,2022,56.1,1.6,1.1
4263,2023-02-24,Away,D,1.0,1.0,FUL,0.8,0.5,49.0,WOL,...,24.0,38.0,35.0,64.6,1.6,0.8,2022,68.9,1.8,0.8
4280,2023-03-01,Away,L,0.0,2.0,LIV,0.8,2.5,43.0,WOL,...,24.0,39.0,35.0,64.8,1.6,0.8,2022,82.9,2.5,0.7


In [166]:
if not os.getcwd().endswith('Football Forecasting Version 2'):
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

os.chdir(fr'{dir_path}\{current_season}_{current_season + 1}_Clean_Data')

today = date.today()

df_final.to_csv(fr'{current_season}_{current_season + 1}_{today.strftime("%b%d")}_historical_data.csv')
